In [2]:
import nltk
import random
import numpy as np
import pandas as pd
import pprint, time
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

'''
Reading the input file and storing the 
values of the 3 columns of each row in
a tuple: (<Word>, <POS_TAG>, <CHUNK_TAG>)
'''
f = open("train.txt", "r")
sentence_corpus = []
sentence = []

for line in f:
    line = line.strip()
    if line == "":
        sentence_corpus.append(sentence)
        sentence = []
    else:
        word, pos_tag, _ = line.split(" ")
        #ignoring the chunk tag for this task
        sentence.append((word, pos_tag))
f.close()

# Add the last sentence (if any)
if sentence:
    sentence_corpus.append(sentence)

In [3]:
#Splitting the corpus data into train_data and test_data (validadtion) (80/20 split)
train_set,test_set =train_test_split(sentence_corpus,train_size=0.80,test_size=0.20,random_state = 101)

# List of all the tags in the train and the test set (it may not be unique)
train_tag_corpus = [ t for sentence in train_set for t in sentence ]
test_tag_corpus = [ t for sentence in test_set for t in sentence ]
    
# Finding number of unique tags and words (Vocabulary)
train_tag_set = {tag for word, tag in train_tag_corpus}
vocab = {word for word, tag in train_tag_corpus}

In [4]:
'''
First implementation: Vertebi Algorithm from scratch
Note: Time consuming: Test data running for more than
      3 Hours.
'''
#Methods to compute transition and emission

'''
prev_tag -> current_tag 
Pr(current_tag | prev_tag) = (# of prev_tag -> current_tag)/(# of prev_tag)
'''
def computeTransition(prev_tag, current_tag):
    tags = [tag for _, tag in train_tag_corpus]
    
    #Count of prev_tag
    cnt_prev_tag = len([tag for tag in tags if tag == prev_tag])
    cnt_prev_curr_tag = 0
    
    for i in range(1, len(tags)):
        if tags[i-1] == prev_tag and tags[i] == current_tag:
            cnt_prev_curr_tag += 1
    
    return cnt_prev_curr_tag / cnt_prev_tag

# compute Emission Probability
def computeEmission(word, tag):
    data = train_tag_corpus
    tag_list = [tg for tg in data if tg[1]==tag]
    cnt_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
 
    return count_w_given_tag / cnt_tag


#The crux of HMM is the emission and transition probabilities

#Transition
transition = np.zeros((len(train_tag_set), len(train_tag_set)), dtype='float32')
train_tag_list = list(train_tag_set)
for i in range(len(train_tag_list)):
    for j in range(len(train_tag_list)):
        transition[i,j] = computeTransition(train_tag_list[i], train_tag_list[j])

In [5]:
#Converting transition matrix into a dataframe for better access of the data.
tags_df = pd.DataFrame(transition, columns = list(train_tag_list), index=list(train_tag_list))

In [6]:
# This part of code helps in cases of unseen words. 
# Using Regex to match the unseen words with the most appropriate tags  

regex = [
    (r'.*ing$', 'VBG'),
    (r'.*ed$', 'VBD'), 
    (r'.*es$', 'VBZ'), 
    (r'.*end$', 'VB'),
    (r'^[A-Z].*$', 'NNP'),
    (r'\b\w+s\b', 'NNS'),
    (r'\b\w+NN\b', 'NN'),
    (r'.*', 'NN'),
    (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),
]

unseen_word_tagger = nltk.RegexpTagger(regex)

In [22]:
#Implmenting HMM: Vertebi 
def Viterbi(words):
    data = train_tag_corpus
    pos = []
    dp = {}
    tags = list(set([pair[1] for pair in data]))
     
    for key, word in enumerate(words):
        prob_lst = []
        prob_max = -float("inf")
        prob_index = 0
        for i, tag in enumerate(tags):
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[pos[-1], tag]
            
            if (tag, word) in dp.keys():
                emission_p = dp[(tag, word)]
            else:
                emission_p = computeEmission(words[key], tag)
                dp[(tag, word)] = emission_p
                
            prob = emission_p * transition_p    
            prob_lst.append(prob)
            if prob > prob_max:
                prob_max = prob
                prob_index = i
             
        # getting state for which probability is maximum
        if not prob_max:
            best_pos_tag = unseen_word_tagger.tag([word])[0][1]
        else:
            best_pos_tag = tags[prob_index] 
        pos.append(best_pos_tag)
    return list(zip(words, pos))

In [ ]:
#Testing random 10 sentences of the dev data.
rndom = [random.randint(1,len(test_set)) for x in range(100)]
test_run = [test_set[i] for i in rndom]
test_run_base = [tup for sent in test_run for tup in sent]
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

get_tag = Viterbi(test_tagged_words)
  
# accuracy
acc = [i for i, j in zip(get_tag, test_run_base) if i == j] 
accuracy = len(acc)/len(get_tag)
print('Viterbi Accuracy: ',accuracy*100)

In [ ]:
##print(get_tag[:5], test_run_base[:5])

In [ ]:
#Testing the dev set
test_tagged_words = [tup for sent in test_set for tup in sent]
test_untagged_words = [tup[0] for sent in test_set for tup in sent]
test_set_base = [tup for sent in test_set for tup in sent]

get_tag = Viterbi(test_untagged_words)
  
# accuracy
acc = [i for i, j in zip(get_tag, test_set_base) if i == j] 
accuracy = len(acc)/len(get_tag)
print('Viterbi Accuracy: ',accuracy*100)

In [9]:
f_test_data = open("test_data.txt", "r")
test_corpus = []

for line in f_test_data:
    line = line.strip()
    if line == "":
        continue
    else:
        test_corpus.append(line)
f_test_data.close()

In [23]:
'''
Testing the test data
'''
# '''
# Testing Vertebi (Warning: Takes a really long time!!!)
# '''
get_tag = Viterbi(test_corpus)

In [ ]:
predicted_pos_tag = [ tag for _, tag in get_tag ]